In [1]:
import json
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tqdm
from skimage.feature import ORB
from skimage.color import rgb2gray
import os
import random

# Load data

In [2]:
# map
with open("Data/database/database.json","r") as f:
    m_idx = json.load(f)
    m_imgs = np.array(m_idx["im_paths"])
    m_loc=np.array(m_idx["loc"])

# query
with open("Data/query/query.json","r") as f:
    q_idx=json.load(f)
    q_imgs=np.array(q_idx["im_paths"])
    q_loc=np.array(q_idx["loc"])

# loading the relevance judgements
with h5py.File("Data/london_gt.h5","r") as f:
    fovs = f["fov"][:]
    sim = f["sim"][:].astype(np.uint8)


# Preprocessing

## Create feature vectors for images

In [3]:
def get_descriptors(x_imgs, n_keypoints=10):
    
    # Initialize the ORB descriptor
    descriptor_extractor = ORB(n_keypoints=n_keypoints)
    # Initialize the data structure that will contain all the descriptors
    descriptors = None

    for img_name in x_imgs:
        #img = Image.open(os.path.join('data_image_retrieval/', img_name)).convert()
        #img = np.asarray(img)
        img = plt.imread(os.path.join('Data/', img_name))
        img = rgb2gray(img)

        # Extract ORB descriptors
        descriptor_extractor.detect_and_extract(img)

        descriptors_img = descriptor_extractor.descriptors  # descriptors (the feature vectors)

        if descriptors is None:
            descriptors = descriptors_img
        else:
            descriptors = np.vstack( (descriptors, descriptors_img))

    return descriptors.reshape((len(x_imgs), -1))

In [4]:
def save_dict(file_path, imgs, descriptors):
    np.save(file_path+"img_path.npy", imgs)
    np.save(file_path+"descriptors.npy", descriptors)

In [5]:
def load_dict(file_path):
    imgs = np.load(file_path+"img_path.npy")
    descriptors = np.load(file_path+"descriptors.npy")

    return dict(zip(imgs, descriptors))

In [6]:
def get_feture_dict(m_imgs, q_imgs, n_keypoints=10):
    m_descriptors = get_descriptors(m_imgs, n_keypoints=n_keypoints)
    q_descriptors = get_descriptors(q_imgs, n_keypoints=n_keypoints)
    
    descriptors = np.concatenate((m_descriptors, q_descriptors))
    
    imgs = np.concatenate((m_imgs, q_imgs))

    save_dict('./Data/Dataset/', imgs, descriptors)

    return dict(zip(imgs, descriptors))

In [7]:
n_keypoints = 10

# Create dictionary for mapping image path to the vector of keypoints
img2vector = get_feture_dict(m_imgs, q_imgs, n_keypoints=n_keypoints)

In [11]:
img2vector = load_dict('./Data/Dataset/')

In [15]:
file_path = q_imgs[random.randint(0,len(q_imgs)-1)]

img2vector[file_path]

array([ True,  True,  True, ..., False, False,  True])

## Create data pairs

In [30]:
X_raw = []
Y = []

for i in range(sim.shape[0]):
    for j in range(sim.shape[1]):
        X_raw.append((q_imgs[i],m_imgs[j]))
        Y.append(sim[i][j])


X_raw = np.array(X_raw)
Y = np.array(Y)

# Find the indices of non-zero values in Y
non_zero_indices = np.nonzero(Y)[0]

# Shuffle the indices to avoid any ordering bias
np.random.shuffle(non_zero_indices)

# Count the number of non-zero values
n_non_zero = np.count_nonzero(Y)

# Filter the zero indices from Y
zero_indices_0 = np.where(Y == 0)[0][:n_non_zero]
zero_indices_1 = np.where(Y == 0)[0][n_non_zero:2*n_non_zero]
zero_indices_2 = np.where(Y == 0)[0][2*n_non_zero:3*n_non_zero]
zero_indices_3 = np.where(Y == 0)[0][3*n_non_zero:4*n_non_zero]
zero_indices_4 = np.where(Y == 0)[0][4*n_non_zero:5*n_non_zero]
zero_indices_5 = np.where(Y == 0)[0][5*n_non_zero:6*n_non_zero]
zero_indices_6 = np.where(Y == 0)[0][6*n_non_zero:7*n_non_zero]
zero_indices_7 = np.where(Y == 0)[0][7*n_non_zero:8*n_non_zero]
zero_indices_8 = np.where(Y == 0)[0][8*n_non_zero:9*n_non_zero]
zero_indices_9 = np.where(Y == 0)[0][9*n_non_zero:10*n_non_zero]

# Concatenate the non-zero and zero indices
balanced_indices = np.concatenate([non_zero_indices, zero_indices_9])

# Use the balanced indices to create new X and Y arrays
X_raw = X_raw[balanced_indices]
Y = Y[balanced_indices]

print(X_raw.shape)
print(Y.shape)

(36106, 2)
(36106,)


## Split trainging, test and validation datasets

In [31]:
X = np.asarray([(img2vector[q_img], img2vector[m_img]) for q_img, m_img in X_raw])

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_rest, y_train, y_rest = train_test_split(X, Y, test_size=0.20, random_state=0)
X_test, X_validate, y_test, y_validate = train_test_split(X_rest, y_rest, test_size=0.50, random_state=0)

print(X_train.shape, X_test.shape, X_validate.shape)

(28884, 2, 2560) (3611, 2, 2560) (3611, 2, 2560)


In [33]:
np.save("./Data/Dataset/X_train.npy", X_train)
np.save("./Data/Dataset/X_test.npy", X_test)
np.save("./Data/Dataset/X_validate.npy", X_validate)

np.save("./Data/Dataset/Y_train.npy", y_train)
np.save("./Data/Dataset/Y_test.npy", y_test)
np.save("./Data/Dataset/Y_validate.npy", y_validate)